# send2vec, word2vect 테스트
- How to Compute Sentence Similarity Using BERT and Word2Vec
    - https://towardsdatascience.com/how-to-compute-sentence-similarity-using-bert-and-word2vec-ab0663a5d64

In [35]:
##################################
# sent2vec 라이브러리
##################################

from sent2vec.vectorizer import Vectorizer
from scipy import spatial

def sim_sentences_sent2vec(sentences):
    vectorizer = Vectorizer()
    print(sentences)
    vectorizer.bert(sentences)
    vectors = vectorizer.vectors    
    dist_1 = spatial.distance.cosine(vectors[0], vectors[1])
    # dist_2 = spatial.distance.cosine(vectors[0], vectors[2])
    # print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))
    print('dist_1: {0}'.format(dist_1))    
    # dist_1: 0.043, dist_2: 0.192

    
##################################
# KoNLpy 토큰라이저
##################################
from konlpy.tag import Okt
Okt = Okt()
pretrained_kr_word2vec = 'resource/ko.bin'
# datapath = 'ko.bin'
datapath = pretrained_kr_word2vec
# print(datapath)
 
import gensim
import os
import numpy as np

word2vec = gensim.models.Word2Vec.load(datapath)

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,형용사, 부사, 알파벳 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Adjective", "Adverb"], stopword=[]):
    return [
        word for word, tag in Okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

    
##################################
# Kakao 개발자 Word2Vec 관련 라이브러리
##################################

    
def sim_sentences_word2vec(sentences):
    '''
    word2vect의 리터하는 벡터들의 평균을 구하고, 이를 코사인 거리를 구함
    '''
    def get_word2vec(word2vec, words, embedding_size=200):
        '''
        단어 리스트를 받고, 벡터로 변환후에 모든 벡터의 평균을 리턴
        '''
        vectors = []
        for i, word in enumerate(words):
            try:
                vector = word2vec[word]    # 워드 -> 벡터
                # print(word)
            except:
                # print("Unknowd words")
                vector = np.zeros(embedding_size) # 모르는 단어이면 0으로 채움
            # break
            vectors.append(vector)
            # print(vector.sum())     

        avg_vectors = np.mean(vectors, axis=0)    
        return avg_vectors
    
    s1 = sentences[0]
    s2 = sentences[1]
    ps1 = tokenizer(s1)
    ps2 = tokenizer(s2)

    avg_vector1 = get_word2vec(word2vec, ps1)
    avg_vector2 = get_word2vec(word2vec, ps2)
    # print(avg_vector)
    dist_w2v = spatial.distance.cosine(avg_vector1, avg_vector2)
    print(s1,': ', ps1)
    print(s1,': ', ps2)    
    print('dist_w2v: {}'.format(dist_w2v))    

##################################
# FastText Word2Vec 관련 라이브러리
##################################
    
from gensim import models
def get_fasttext_model(path):
    '''
    path = 'resource/cc.ko.300.bin'
    ko_model = get_fasttext_model(path)    
    '''    
    try:
        if type(ko_model) == gensim.models.fasttext.FastText:
            print('Model is already loaded')
        else:
            print('Model is loading')
            ko_model = models.fasttext.load_facebook_model(path)
            
    except:
        print('Model is loading')
        ko_model = models.fasttext.load_facebook_model(path)
        
    return ko_model        
    
def sim_sentences_fasttext_word2vec(ko_model, sentences):
    '''
    word2vect의 리터하는 벡터들의 평균을 구하고, 이를 코사인 거리를 구함
    '''
    def get_word2vec(ko_model, words, embedding_size=300):
        '''
        단어 리스트를 받고, 벡터로 변환후에 모든 벡터의 평균을 리턴
        '''
        vectors = []
        for i, word in enumerate(words):
            try:
                vector = ko_model.wv.get_vector(word)
                # print(word)
            except:
                # print("Unknowd words")
                vector = np.zeros(embedding_size) # 모르는 단어이면 0으로 채움
            # break
            vectors.append(vector)
            # print(vector.sum())     

        avg_vectors = np.mean(vectors, axis=0)    
        return avg_vectors
    
    s1 = sentences[0]
    s2 = sentences[1]
    ps1 = tokenizer(s1)
    ps2 = tokenizer(s2)

    avg_vector1 = get_word2vec(ko_model, ps1)
    avg_vector2 = get_word2vec(ko_model, ps2)
    # print(avg_vector)
    dist_w2v = spatial.distance.cosine(avg_vector1, avg_vector2)
    print(s1,': ', ps1)
    print(s1,': ', ps2)    

    print('dist_w2v: {}'.format(dist_w2v))    
    


## 테스트 문장

In [28]:
s1 ='사과는 과일이다'
s2 = '책은 인류가 쌓은 지식의 보고다'
s3 = '건강에 좋은 것은 과일이다'
# s3 ='사과는 과일이다'

sents1 = [s1, s2]    
sents2 = [s1, s3]    

## sent2vec 테스트
0 에 가까울 수록 유사한 것을 의미함

In [29]:
sim_sentences_sent2vec(sents1)
sim_sentences_sent2vec(sents2)

['사과는 과일이다', '책은 인류가 쌓은 지식의 보고다']
dist_1: 0.028812527656555176
['사과는 과일이다', '건강에 좋은 것은 과일이다']
dist_1: 0.009791731834411621


## Kakao Word2Vec 테스트

In [30]:
sim_sentences_word2vec(sents1)
sim_sentences_word2vec(sents2)


사과는 과일이다 :  ['사과', '과일']
사과는 과일이다 :  ['인류', '쌓다', '지식', '보고']
dist_w2v: 0.8379042084884104
사과는 과일이다 :  ['사과', '과일']
사과는 과일이다 :  ['건강', '좋다', '과일']
dist_w2v: 0.3584424536921966


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## FastText Word2Vec 테스트

In [44]:
try:
    ko_model == gensim.models.fasttext.FastText
except:
    print("Need to load a model")
    path = 'resource/cc.ko.300.bin'    
    ko_model = get_fasttext_model(path)    

sim_sentences_fasttext_word2vec(ko_model, sents1)
sim_sentences_fasttext_word2vec(ko_model, sents2)


사과는 과일이다 :  ['사과', '과일']
사과는 과일이다 :  ['인류', '쌓다', '지식', '보고']
dist_w2v: 0.680292159318924
사과는 과일이다 :  ['사과', '과일']
사과는 과일이다 :  ['건강', '좋다', '과일']
dist_w2v: 0.3414410352706909
